# Handling Event Data
*by: Sebastiaan J. van Zelst*

Process mining exploits Event Logs to generate knowledge of a process.
A wide variety of information systems, e.g., SAP, ORACLE, SalesForce, etc., allow us to extract, in one way or the other,
event logs similar to the example event logs.
All the examples we show in this notebook and all algorithms implemented in pm4py assume that we have already extracted
the event data into an appropriate event log format.
Hence, the core of pm4py does not support any data extraction features.

In order to support interoperability between different process mining tools and libraries, two standard data formats are
used to capture event logs, i.e., Comma Separated Value (CSV) files and eXtensible Event Stream (XES) files.
CSV files resemble the example tables shown in the previous section, i.e., Table 1 and Table 2. Each line in such a file
describes an event that occurred. The columns represent the same type of data, as shown in the examples, e.g., the case
for which the event occurred, the activity, the timestamp, the resource executing the activity, etc.
The XES file format is an XML-based format that allows us to describe process behavior.
We will not go into specific details w.r.t. the format of XES files, i.e., we refer to http://xes-standard.org/ for an
overview.

In this tutorial, we will use an oftenly used dummy example event log to explain the basic process mining operations.
The process that we are considering is a simplified process related to customer complaint handling, i.e., taken from the
book of van der Aalst (https://www.springer.com/de/book/9783662498507). The process, and the event data we are going to
use, looks as follows.

![Running example BPMN-based process model describing the behavior of the simple process that we use in this tutorial](img/bpmn_running_example.png)

## Importing CSV Files

Let’s get started!
We have prepared a small sample event log, containing behavior similar equal to the process model in Figure 3.
You can find the sample event log [here](data/running_example.csv).

We are going to load the event data, and, we are going to count how many cases are present in the event log, as well as
the number of events. Note that, for all this, we are effectively using a third-party library called pandas.
We do so because pandas is the de-facto standard of loading/manipulating csv-based data.
Hence, any process mining algorithm implemented in pm4py, using an event log as an input, can work directly with a
pandas file!


In [1]:
import pandas as pd

df = pd.read_csv('data/running_example.csv', sep=';')
df

,case_id,activity,timestamp,costs,org:resource
0,3,register request,2010-12-30 14:32:00+01:00,50,Pete
1,3,examine casually,2010-12-30 15:06:00+01:00,400,Mike
2,3,check ticket,2010-12-30 16:34:00+01:00,100,Ellen
3,3,decide,2011-01-06 09:18:00+01:00,200,Sara
4,3,reinitiate request,2011-01-06 12:18:00+01:00,200,Sara
5,3,examine thoroughly,2011-01-06 13:06:00+01:00,400,Sean
6,3,check ticket,2011-01-08 11:43:00+01:00,100,Pete
7,3,decide,2011-01-09 09:55:00+01:00,200,Sara
8,3,pay compensation,2011-01-15 10:45:00+01:00,200,Ellen
9,2,register request,2010-12-30 11:32:00+01:00,50,Mike


Let's inspect the small event log.
The first line (i.e., row) specifies the name of each column (i.e., event attribute).
Observe that, in the data table described by the file, we have 5 columns, being: *case_id*, *activity*,
*timestamp*, *costs* and *org:resource*.
The first column represents the *case identifier*, i.e., allowing us to identify what activity has been logged in the
context of what instance of the process.
The second column (*activity*) records the activity that has been performed.
The third column shows at what point in time the activity was recorded (*timestamp*).
In this example data, additional information is present as well.
In this case, the fourth column tracks the costs of the activity (*costs* attribute), whereas the fifth row tracks what
resource has performed the activity (*org:resource*).

Observe that, row 2-10 show the events that have been recorded for the process identified by *case identifier* 3.
We observe that first a register request activity was performed, followed by the examine casually, check ticket, decide,
reinitiate request, examine thoroughly, check ticket,decide, and finally, pay compensation activities.
Note that, in this case, the recorded process instance behaves as described by the model depicted in Figure 3.

Let's investigate some basic statistics of our log, e.g., the total number of cases described and the total number of events.

In [3]:
# number of cases
len(df['case_id'].unique())

6

In [4]:
# number of events
len(df)


42

### Formatting Data Frames

Now we have loaded our first event log, it is time to put some pm4py into the mix.
pm4py uses standardized column names to represent the *case identifier*, the *activity name* and the timstamp.
These are, respectively, ```case:concept:name```, ```concept:name``` and ```time:timestamp```.
Hence, to make pm4py work with the provided csv file, we need to rename the ```case_id```, ```activity``` and ```timestamp``` columns.
pm4py provides a dedicated utility function for this:

In [5]:
import pm4py
log = pm4py.format_dataframe(df, case_id='case_id',activity_key='activity',
                             timestamp_key='timestamp')
log


,case:concept:name,concept:name,time:timestamp,costs,org:resource,@@index
14,1,register request,2010-12-30 10:02:00+00:00,50,Pete,14
15,1,examine thoroughly,2010-12-31 09:06:00+00:00,400,Sue,15
16,1,check ticket,2011-01-05 14:12:00+00:00,100,Mike,16
17,1,decide,2011-01-06 10:18:00+00:00,200,Sara,17
18,1,reject request,2011-01-07 13:24:00+00:00,200,Pete,18
9,2,register request,2010-12-30 10:32:00+00:00,50,Mike,9
10,2,check ticket,2010-12-30 11:12:00+00:00,100,Mike,10
11,2,examine casually,2010-12-30 13:16:00+00:00,400,Sean,11
12,2,decide,2011-01-05 10:22:00+00:00,200,Sara,12
13,2,pay compensation,2011-01-08 11:05:00+00:00,200,Ellen,13


Observe that the column names are updated as expected.

Let us assume that we are not only interested in the number of events and cases, yet, we also want to figure out what
activities occur first, and what activities occur last in the traces described by the event log.
pm4py has a specific built-in function for this, i.e., ```pm4py.get_start_activities()``` and ```pm4py.get_end_activities()``` respectively.

In [6]:
pm4py.get_start_activities(log)

{'register request': 6}

In [7]:
pm4py.get_end_activities(log)

{'pay compensation': 3, 'reject request': 3}

The ```pm4py.get_start_activities()``` and ```pm4py.get_end_activities()``` both return a dictionary containing the activities
as a key, and, the number of observations (i.e., number of traces in which they occur first, respectively, last) in
the event log.

pm4py exploits a built-in pandas function to detect the format of the timestamps in the input data automatically.
However, pandas looks at the timestamp values in each row in isolation.
In some cases, this can lead to problems.
For example, if the provided value is 2020-01-18, i.e., first the year, then the month, and then the day of the date,
in some cases, a value of 2020-02-01 may be interpreted wrongly as January 2nd, i.e., rather than February 1st.
To alleviate this problem, an additional parameter can be provided to the ```format_dataframe()``` method, i.e.,
the timest_format parameter. The default Python timestamp format codes can be used to provide the timestamp format.
In this example, the timestamp format is ```%Y-%m-%d %H:%M:%S%z```.
In general, we advise to always specify the timestamp format.

## Importing XES Files

Next to CSV files, event data can also be stored in an XML-based format, i.e., in XES files.
In an XES file, we can describe a containment relation, i.e., a log contains a number of traces, which in turn contain several events.
Furthermore, an object, i.e., a log, trace, or event, is allowed to have attributes.
The advantage is that certain data attributes that are constant for a log or a trace, can be stored at that level.
For example, assume that we only know the total costs of a case, rather than the costs of the individual events.
If we want to store this information in a CSV file, we either need to replicate this information (i.e., we can only
store data in rows, which directly refer to events), or, we need to explicitly define that certain columns only get a
value once, i.e., referring to case-level attributes.
The XES standard more naturally supports the storage of this type of information.
Click [here](data/running_example.xes) to obtain the .xes file of the running_example.

Importing an XES file is fairly straightforward.
pm4py has a special read_xes()-function that can parse a given xes file and load it in pm4py, i.e., as an Event Log object.
Consider the following code snippet, in which we show how to import an XES event log.
Like the previous example, the script outputs activities that can start and end a trace.

In [12]:
log_xes = pm4py.read_xes('data/running_example.xes', return_legacy_log_object=True)
pm4py.get_start_activities(log_xes)

parsing log, completed traces ::   0%|          | 0/6 [00:00<?, ?it/s]

{'register request': 6}

In [13]:
pm4py.get_end_activities(log_xes)


{'pay compensation': 3, 'reject request': 3}

## Exporting Event Data

Now we have seen how to import event data into pm4py, let’s take a look at the opposite, i.e., exporting event data.
Exporting of event logs can be very useful, e.g., we might want to convert a .csv file into a ```.xes``` file or we might
want to filter out certain (noisy) cases and save the filtered event log. Like importing, exporting of event data is
possible in two ways, i.e., exporting to ```csv``` (using ```pandas```) and exporting event logs to xes. In the upcoming
sections, we show how to export an event log stored as a ```pandas data frame``` into a ```csv``` file, a ```pandas data frame``` as an
```xes file```, a pm4py ```event log object``` as a ```csv file``` and finally, a pm4py ```event log object``` as an ```xes file```.

### Storing a Pandas Data Frame as a csv file

Storing an event log that is represented as a pandas dataframe is straightforward, i.e., we can directly use the ```to_csv```
 ([full reference here](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_csv.html)) function
 of the pandas DataFrame object. Consider the following example snippet of code, in which we show this functionality.

In [14]:
log.to_csv('running_example_exported.csv')

### Storing a Pandas DataFrame as a .xes file

It is also possible to store a pandas data frame to a xes file. This is simply done by calling the ```pm4py.write_xes()```
function. You can pass the dataframe as an input parameter to the function, i.e., pm4py handles the internal conversion
of the dataframe to an event log object prior to writing it to disk. Note that this construct only works if you have
formatted the data frame, i.e., as highlighted earlier in the importing CSV section.

In [ ]:
pm4py.write_xes(log, 'running_example_csv_exported_as_xes.xes')


### Storing an Event Log object as a .csv file

In some cases, we might want to store an event log object, e.g., obtained by importing a .xes file, as a csv file.
For example, certain (commercial) process mining tools only support csv importing. 
For this purpose, pm4py offers conversion functionality that allows you to convert your event log object into a data frame,
which you can subsequently export using pandas.


In [16]:
df = pm4py.convert_to_dataframe(log_xes)
df.to_csv('running_example_xes_exported_as_csv.csv')


### Storing an Event Log Object as a .xes File

Storing an event log object as a .xes file is rather straightforward. In pm4py, the write_xes() method allows us to do so.
Consider the simple example script below in which we show an example of this functionality.

In [ ]:
pm4py.write_xes(log_xes, 'running_example_exported.xes')